### 311 Calls in NYC

Note: a smaller version of the dataset has been provided. Make sure to have it downloaded in the cwd.

If you want to download the full dataset (~41M rows) you can do it [here](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9/about_data)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (15, 5)

complaints = pd.read_csv('311_small.csv')  # we'll use the small version here